# <font color=blue>Декораторы (элемент синтаксиса)</font>

Декораторы в Python и примеры их практического использования.

Итак, что же это такое? Для того, чтобы понять, как работают декораторы, в первую очередь следует вспомнить, что функции в python являются объектами, соответственно, их можно возвращать из другой функции или передавать в качестве аргумента. Также следует помнить, что функция в python может быть определена и внутри другой функции.

Вспомнив это, можно смело переходить к декораторам. Декораторы — это, по сути, "обёртки", которые дают нам возможность изменить поведение функции, не изменяя её код.

Создадим свой декоратор "вручную":

In [11]:
def my_shiny_new_decorator(function_to_decorate):
    # Внутри себя декоратор определяет функцию-"обёртку". Она будет обёрнута вокруг декорируемой,
    # получая возможность исполнять произвольный код до и после неё.
    def the_wrapper_around_the_original_function():
        print("Я - код, который отработает до вызова функции")
        function_to_decorate() # Сама функция
        print("А я - код, срабатывающий после")
    # Вернём эту функцию
    return the_wrapper_around_the_original_function

# Представим теперь, что у нас есть функция, которую мы не планируем больше трогать.
def stand_alone_function():
    print("Я простая одинокая функция, ты ведь не посмеешь меня изменять?")

stand_alone_function()
# Однако, чтобы изменить её поведение, мы можем декорировать её, то есть просто передать декоратору,
# который обернет исходную функцию в любой код, который нам потребуется, и вернёт новую,
# готовую к использованию функцию:
stand_alone_function_decorated = my_shiny_new_decorator(stand_alone_function)
stand_alone_function_decorated()

Я простая одинокая функция, ты ведь не посмеешь меня изменять?
Я - код, который отработает до вызова функции
Я простая одинокая функция, ты ведь не посмеешь меня изменять?
А я - код, срабатывающий после


Возможно мы бы хотели, чтобы каждый раз, во время вызова `stand_alone_function()`, вместо неё вызывалась `stand_alone_function_decorated()`. Для этого просто перезапишем `stand_alone_function()`:

In [12]:
stand_alone_function = my_shiny_new_decorator(stand_alone_function)
stand_alone_function()

Я - код, который отработает до вызова функции
Я простая одинокая функция, ты ведь не посмеешь меня изменять?
А я - код, срабатывающий после


Собственно, это и есть декораторы. Вот так можно было записать предыдущий пример, используя синтаксис декораторов:

In [ ]:
@my_shiny_new_decorator
def another_stand_alone_function():
    print("Оставь меня в покое")

another_stand_alone_function()

То есть, декораторы в python — это просто синтаксическая обертка для конструкций вида:

In [ ]:
def another_stand_alone_function():
    print("Оставь меня в покое")

another_stand_alone_function = my_shiny_new_decorator(another_stand_alone_function)

Можно использовать несколько декораций для функций:

In [13]:
def bread(func):
    def wrapper():
        print()
        func()
        print("<\______/>")
    return wrapper

def ingredients(func):
    def wrapper():
        print("#помидоры#")
        func()
        print("~салат~")
    return wrapper

def sandwich(food="--ветчина--"):
    print(food)

sandwich()
sandwich = bread(ingredients(sandwich))
sandwich()

--ветчина--

#помидоры#
--ветчина--
~салат~
<\______/>


И аналогично через декораторы:

In [ ]:
@bread
@ingredients
def sandwich(food="--ветчина--"):
    print(food)

sandwich()

Не забываем, что так как порядок вызова функций имеет значение, то и порядок проставление декораторов так же имеет значение.

##  Упражнение 1

Напишите функцию, которая получает на вход список чисел и выдает ответ сколько в данном списке четных чисел. Напишите декоратор, который меняет поведение функции следующим образом: если четных чисел нет, то пишет "Нету(" а если их больше 10, то пишет "Очень много"

## <font color=green>Передача декоратором аргументов в функцию</font>

Однако, все декораторы, которые мы рассматривали, не имели одного очень важного функционала — передачи аргументов декорируемой функции. Собственно, это тоже несложно сделать.

Текстовый данные в языке пайтон описываются классом `str`:

In [14]:
def a_decorator_passing_arguments(function_to_decorate):
    def a_wrapper_accepting_arguments(arg1, arg2):
        print("Смотри, что я получил:", arg1, arg2)
        function_to_decorate(arg1, arg2)
    return a_wrapper_accepting_arguments

# Теперь, когда мы вызываем функцию, которую возвращает декоратор, мы вызываем её "обёртку",
# передаём ей аргументы и уже в свою очередь она передаёт их декорируемой функции
@a_decorator_passing_arguments
def print_full_name(first_name, last_name):
    print("Меня зовут", first_name, last_name)

print_full_name("Vasya", "Pupkin")

Смотри, что я получил: Vasya Pupkin
Меня зовут Vasya Pupkin


# <font color=green>Декорирование методов</font>

Один из важных фактов, которые следует понимать, заключается в том, что функции и методы в Python — это практически одно и то же, за исключением того, что методы всегда ожидают первым параметром ссылку на сам объект (`self`). Это значит, что мы можем создавать декораторы для методов точно так же, как и для функций, просто не забывая про `self`.

При этом строка представляет из себя объект-коллекцию и есть возможность получить доступ к отдельным ее элементам по индексу:

In [15]:
def method_friendly_decorator(method_to_decorate):
    def wrapper(self, lie):
        lie -= 3
        return method_to_decorate(self, lie)
    return wrapper

class Lucy:
    def __init__(self):
        self.age = 32
    @method_friendly_decorator
    def sayYourAge(self, lie):
        print("Мне {} лет, а ты бы сколько дал?".format(self.age + lie))

l = Lucy()
l.sayYourAge(-3)

Мне 26 лет, а ты бы сколько дал?


### Упражнение 2

Воспользуйтесь написанным классом `Vector2D` и методом `__add__()`. Добавьте к нему декоратор, который при вызове метода печатает сообщение вида: (1, 2) + (3, -1) = (2, 1)

In [ ]:



class Vector2D:
    def __init__(self, *args):
        self.set_xy(*args)
            
    def get_xy(self):
        return [self._x, self._y]
    
    def set_xy(self, *args):
        if len(args) == 2:
            self._x = args[0]
            self._y = args[1]
        elif len(args) == 1:
            self._x = args[0][0]
            self._y = args[0][1]
        else:
            raise ValueError("wrong number of arguments. len(args) == {}".format(len(args)))
            
    def __add__(self, other):
        x, y = other.get_xy()
        return Vector2D(self._x + x, self._y + y)
    
    def __repr__(self):
        return self.__class__.__name__ + '([' + ', '.join(map(str, [self._x, self._y])) + '])'
    
a = Vector2D(1, 2)
b = Vector2D(3, 4)
c = a + b
print(a)
print(b)
c